In [1]:
from pathlib import Path
import json
import pandas as pd
from collections import Counter, defaultdict

DATA_DIR = Path("/Users/miller/projects/File_Util_App/output/sections/item_20")

# Guard clause: bail out early if the path is wrong.
if not DATA_DIR.exists(): raise FileNotFoundError(f"{DATA_DIR} not found")

In [4]:
def load_item20_tables(path: Path) -> dict[str, pd.DataFrame]:
    """Return {'t1': df, 't2': df, …} for a single Item 20 JSON."""
    with path.open() as f:
        raw = json.load(f)

    dataframes = {}
    for key, value in raw.items():
        rows = value.get("rows", [])
        columns = value.get("columns", [])

        if rows:
            # Check if all rows have the same number of items as the first row.
            # This is a common precursor to DataFrame creation issues.
            first_row_len = len(rows[0])
            if not all(len(row) == first_row_len for row in rows):
                print(f"Warning: File {path}, table '{key}' has rows with inconsistent numbers of items. "
                      f"The first row has {first_row_len} items. Please inspect the data. "
                      f"Attempting to create DataFrame, but it may fail or produce unexpected results.")
                # Attempt to create DataFrame; pandas might raise an error if lengths are too different
                try:
                    df = pd.DataFrame(rows)
                    # If successful, check if provided column headers match inferred ones
                    if len(df.columns) != len(columns):
                         print(f"Warning: Further column mismatch for table '{key}' in {path} after handling inconsistent rows. "
                               f"Provided headers count: {len(columns)}, inferred data columns: {len(df.columns)}. Using inferred columns.")
                    else:
                         df.columns = columns # Try to apply original column names if counts match
                except ValueError as e:
                    print(f"Error: Could not create DataFrame for table '{key}' in {path} due to inconsistent row lengths: {e}. Creating empty DataFrame.")
                    df = pd.DataFrame(columns=columns if columns else None)

            # If all rows have consistent length, proceed to check against headers
            elif first_row_len != len(columns):
                print(f"Warning: Column mismatch in file {path}, table '{key}'. "
                      f"Expected {len(columns)} columns based on headers, but data rows have {first_row_len} columns. "
                      f"Pandas will use inferred column names for table '{key}'.")
                df = pd.DataFrame(rows)  # Let pandas infer columns
            else:
                df = pd.DataFrame(rows, columns=columns) # Lengths match, proceed as normal
        else:  # No rows
            df = pd.DataFrame(rows, columns=columns) # Create empty DataFrame with headers

        dataframes[key] = df
    return dataframes

In [5]:
header_counts: dict[str, Counter] = defaultdict(Counter)

for fp in DATA_DIR.glob("*.json"):
    for tkey, df in load_item20_tables(fp).items():
        header_counts[tkey].update(df.columns)

# Quick look at the variations:
for tkey, ctr in header_counts.items():
    print(f"\n=== {tkey.upper()} ({len(ctr)} unique headers) ===")
    for col, n in ctr.most_common():
        print(f"{n:>3} × {col}")


=== T1 (157 unique headers) ===
616 × Year
614 × Net Change
589 × Outlet Type
523 × Outlets at the Start of the Year
517 × Outlets at the End of the Year
 38 × Column 1 Outlet Type
 38 × Column 5 Net Change
 36 × Column 2 Year
 36 × Column 3 Outlets at the Start of the Year
 36 × Column 4 Outlets at the End of the Year
 26 × OUTLET TYPE
 25 × YEAR
 22 × OUTLETS AT THE START OF THE YEAR
 22 × OUTLETS AT THE END OF THE YEAR
 21 × Outlets at Start of Year
 20 × Outlets at End of Year
 19 × NET CHANGE
 12 × Outlets at End of the Year
 10 × Outlets at Start of the Year
 10 × Business Type
 10 × 0
 10 × 1
 10 × 2
 10 × 3
 10 × 4
 10 × 5
  8 × Stores at the Start of the Year
  8 × Stores at the End of the Year
  8 × Outlets at the Start of Year
  8 × Outlets at the End of Year
  7 × Net change
  7 × Store Type
  6 × Businesses at the Start of the Year
  5 × Restaurant Type
  5 × Restaurants at the Start of the Year
  5 × Restaurants at the End of the Year
  5 × Businesses at the End of the Y

In [6]:
CANONICAL = {
    # --- t1 -----------------------------------------------------
    "outlet type": "Outlet Type",
    "year": "Year",
    "outlets at the start of the year": "Outlets at the Start of the Year",
    "outlets at the end of the year": "Outlets at the End of the Year",
    "net change": "Net Change",

    # --- t2 -----------------------------------------------------
    "state": "State",
    "number of transfers": "Number of Transfers",

    # --- t3 -----------------------------------------------------
    "outlets at start of year": "Outlets at Start of Year",
    "outlets opened": "Outlets Opened",
    "terminations": "Terminations",
    "non-renewals": "Non-Renewals",
    "reacquired by franchisor": "Reacquired by Franchisor",
    "ceased operations and other reasons": "Ceased Operations – Other Reasons",
    "ceased operations- other reasons": "Ceased Operations – Other Reasons",
    "ceased operations other reasons": "Ceased Operations – Other Reasons",
    "franchised outlets operating at year end": "Outlets at End of Year",
    "franchised stores operating at year end": "Outlets at End of Year",
    "outlets at end of the year": "Outlets at End of Year",

    # --- t4 -----------------------------------------------------
    "outlets closed": "Outlets Closed",
    "outlets reacquired from franchisees": "Outlets Reacquired from Franchisees",
    "outlets sold to franchisees": "Outlets Sold to Franchisees",
    # synonyms/typos ↓
    "outlets reacquired from franchisees": "Outlets Reacquired from Franchisees",
    "outlets sold to franchisees": "Outlets Sold to Franchisees",

    # --- t5 -----------------------------------------------------
    "franchise agreements signed but outlet not open": "Franchise Agreements Signed but Outlet Not Open",
    "franchise agreements signed but facility not opened": "Franchise Agreements Signed but Outlet Not Open",
    "projected new franchised outlets in the next fiscal year": "Projected New Franchised Outlets (Next FY)",
    "projected franchised new facilities in the next fiscal year": "Projected New Franchised Outlets (Next FY)",
    "projected company owned openings in next fiscal year": "Projected New Company-Owned Outlets (Next FY)",
    "projected new company-owned outlets in the current fiscal year": "Projected New Company-Owned Outlets (Current FY)",
}

In [7]:
import re

def canonical(col: str) -> str:
    """Return canonical header or the original if we have no mapping."""
    key = re.sub(r"[^a-z0-9]+", " ", col.lower()).strip()
    return CANONICAL.get(key, col)   # early return style